In [ ]:
# 2 versions of model: 1 where we have transit_ptile, 1 where we have No_of_stops and PTover45min_pct

In [2]:
import pandas as pd
import numpy as np

In [51]:
df = pd.read_csv("DSPH Capstone Data.csv")
df = df.drop(df.loc[:, 'Unnamed: 9':'Unnamed: 23'].columns, axis = 1)
df = df.drop(["GEOID", "PTRAF"],\
             axis = 1)
df

,ct,svi_ptile,transit_ptile,asthmavalue,PTRAF_bycensus,No_of_Stops,PTover45m_pct
0,24510010100,0.04569,0.029276,0.087,685.684371,10,24.51
1,24510010100,0.04569,0.029276,0.087,685.684371,10,24.51
2,24510010100,0.04569,0.029276,0.087,685.684371,10,24.51
3,24510010200,0.01523,0.124807,0.082,686.778083,8,54.24
4,24510010200,0.01523,0.124807,0.082,686.778083,8,54.24
...,...,...,...,...,...,...,...
648,24510280404,0.83250,0.445300,0.124,1527.100404,10,52.66
649,24510280500,0.97970,0.231125,0.136,3385.685542,25,65.99
650,24510280500,0.97970,0.231125,0.136,3385.685542,25,65.99
651,24510280500,0.97970,0.231125,0.136,3385.685542,25,65.99


In [52]:
# get rid of GEOID and PTRAF
for i in df:
    if df['ct'][i] == df['ct'][i-1]:
        df.drop(df['ct'][i])

KeyError: 'ct'

In [48]:
y = df['asthmavalue']


In [ ]:
x1_svi_ptile
x2_transit_ptile